In [1]:
from json import loads
from pandas import DataFrame

FILE_NAME = 'league39_season2020.json'

with open(FILE_NAME) as f:
    l = f.readlines()
    as_str = ''.join(l)
    as_dict = loads(as_str)

response = as_dict['response']

In [2]:
def round_num(round_str: str):
    s = round_str.lstrip("Regular Season - ")
    return int(s)

In [3]:
def round_in_range(round_number):
    return 6 < int(round_number) < 33

In [4]:
all_games = []
games_requested = []
for game in response:
    host = game["teams"]["home"]['name']
    hosted = game["teams"]["away"]['name']
    goals_host = game["goals"]["home"]
    goals_hosted = game["goals"]["away"]

    round_str = game["league"]["round"]
    round_n = round_num(round_str=round_str)

    all_games.append(
        [host, hosted, goals_host, goals_hosted, round_n])
    
    if round_in_range(round_n):
        games_requested.append([host, hosted, goals_host, goals_hosted, round_n])

all_games;

In [5]:
df = DataFrame(games_requested, columns=['host', 'hosted', 'goals_host', 'goals_hosted', 'round'])
df

,host,hosted,goals_host,goals_hosted,round
0,Aston Villa,Southampton,3,4,7
1,Burnley,Chelsea,0,3,7
2,Fulham,West Brom,2,0,7
3,Leeds,Leicester,1,4,7
4,Liverpool,West Ham,2,1,7
...,...,...,...,...,...
255,Leicester,West Brom,3,0,32
256,Manchester United,Burnley,3,1,32
257,Newcastle,West Ham,3,2,32
258,Southampton,Crystal Palace,3,1,32


In [6]:
host_df = df.groupby('host').agg('sum').reset_index()
host_df.drop(labels=['round'], axis=1)

,host,goals_host,goals_hosted
0,Arsenal,15,16
1,Aston Villa,16,16
2,Brighton,12,12
3,Burnley,13,15
4,Chelsea,20,11
5,Crystal Palace,13,22
6,Everton,11,19
7,Fulham,8,15
8,Leeds,17,14
9,Leicester,24,15


In [7]:
hosted_df = df.groupby('hosted').agg('sum').reset_index()
hosted_df.drop(labels=['round'], axis=1)

,hosted,goals_host,goals_hosted
0,Arsenal,13,21
1,Aston Villa,12,16
2,Brighton,14,11
3,Burnley,20,10
4,Chelsea,11,17
5,Crystal Palace,24,13
6,Everton,12,18
7,Fulham,13,12
8,Leeds,27,21
9,Leicester,14,21


In [8]:
results = DataFrame(data={"team": host_df.host,
                          "GF_tot": host_df.goals_host + hosted_df.goals_hosted,
                          "GA_tot": host_df.goals_hosted + hosted_df.goals_host,
                          "GF_as_host": host_df.goals_host,
                          "GA_as_host": host_df.goals_hosted,
                          "GF_as_hosted": hosted_df.goals_hosted,
                          "GA_as_hosted": hosted_df.goals_host,
                          })
results.sort_values(by=['team'], ascending=True)

,team,GF_tot,GA_tot,GF_as_host,GA_as_host,GF_as_hosted,GA_as_hosted
0,Arsenal,36,29,15,16,21,13
1,Aston Villa,32,28,16,16,16,12
2,Brighton,23,26,12,12,11,14
3,Burnley,23,35,13,15,10,20
4,Chelsea,37,22,20,11,17,11
5,Crystal Palace,26,46,13,22,13,24
6,Everton,29,31,11,19,18,12
7,Fulham,20,28,8,15,12,13
8,Leeds,38,41,17,14,21,27
9,Leicester,45,29,24,15,21,14
